# Introduction
League of Legends (LoL) is just one of those games that really frustrating when you lose, but pretty fun overall - even more so with friends!  Here I will try to to map key features that can lead to a win.  Special thanks to [Dr. Penguin](https://www.kaggle.com/xiyuewang/lol-how-to-win) as I modified from his notebook.

In [ ]:
# Load packages and dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%matplotlib inline
sns.set_style('darkgrid')

In [ ]:
df = pd.read_csv('../input/league-of-legends-diamond-ranked-games-10-min/high_diamond_ranked_10min.csv')
df.head()

# EDA

In [ ]:
# check missing values and data type
df.info()

In [ ]:
df.isnull().values.any()

Dataset is pretty clean so, I'm not gonna remove any features for now except for gameId.

In [ ]:
df = df.drop(columns='gameId')
#Checking if removed
df.head()

In [ ]:
# Next let's check the relationship between parameters of blue team features
g = sns.PairGrid(data=df, vars=['blueKills', 'blueAssists', 'blueWardsPlaced', 'blueTotalGold'], hue='blueWins', size=3, palette='Set1')
g.map_diag(plt.hist)
g.map_offdiag(plt.scatter)
g.add_legend();

We can see co-linearity between variables.  I am going to go with a tree based method.  Luckily, tree algorithms are immune to multicollinearity by nature, since the tree will choose only one of the perfectly correlated features.

In [ ]:
# We can see that a lot of the features are highly correlated, let's get the correlation matrix
plt.figure(figsize=(16, 12))
sns.heatmap(df.drop('blueWins', axis=1).corr(), cmap='YlGnBu', annot=True, fmt='.2f', vmin=0);

# Model Selection

In [ ]:
# train test split scale the set
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
X = df.drop(columns='blueWins')
y = df['blueWins']

# I am chosing not to scale the data first, since I am going the tree route.
#Decision trees and ensemble methods do not require feature scaling to be performed as they are not sensitive to the the variance in the data.

##scaler = MinMaxScaler()
##scaler.fit(X)
##X = scaler.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Gradient Boosting

In [ ]:
# fit the model
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn import metrics

gb = GradientBoostingClassifier()

# search the best params
grid = {'n_estimators':[100,200,300,400,500], 'max_depth': [2, 5, 10]}

clf_gb = GridSearchCV(gb, grid, cv=5)
clf_gb.fit(X_train, y_train)

pred_gb = clf_gb.predict(X_test)

# get the accuracy score
acc_gb = accuracy_score(pred_gb, y_test)
print(acc_gb)

In [ ]:
# confusion matrix
from sklearn.metrics import plot_confusion_matrix
# Plot non-normalized confusion matrix
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(clf_gb, X_test, y_test,
                                 #display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()

## Feature Importance for Win

In [ ]:
# Getting settings from GridCV
clf_gb.best_params_

In [ ]:
#inputting best settings from GridCV
gb = GradientBoostingClassifier(n_estimators=100, max_depth=3)
clf_gb.fit(X_train, y_train)

In [ ]:
from sklearn.inspection import permutation_importance

r = permutation_importance(clf_gb, X_test, y_test,
                           n_repeats=20,
                           random_state=0)

for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{X_train.columns[i]:<10}"
              f"{r.importances_mean[i]:.3f}"
              f" +/- {r.importances_std[i]:.3f}")

# Conclusion

Based on the minimally tuned gradient boosted trees, the prediction accuracy is roughly 73%.  I think that's not too bad, since most League win rates hover around 50% anyway.  Based on the feature importance, we can see that acquiring gold is the largest benefit to winning a game.  But that really isn't too suprising, since gold is an indirect measure of how well a team is doing.  

The next big advantage is getting dragons.  Now this makes sense due to dragons giving permanent buffs. Also having good dragon control from my experience means that you have a good jungler with a strong enough bot lane to secure the buff.  Closely tied to dragons is the jungle minions killed.  I think this makes a sense too due to a stronger jungler tends to have better items that lead to more successful ganks.

The last key feature I want to discuss is vision.  Vision is key, since if you know where the enemy is then you can make proper plans to avoid deaths and plan for objectives.

In summary, LoL is a pretty complex game and, based on this simple analysis, that there are many factors that come into play for a victory.  If I had to sum it up in three points to help you win more games is get dragons, CS well, and maintain vision control to reduce deaths.  Hope you liked it for my first shared kernel/notebook and comments are welcome!